# Trabalho de AEDV (A1)
    Igor Cortes Junqueira
    Igor Patricio Michels

Preparação do ambiente:

In [1]:
library(readr)
library(stringr)
library(rlist)
library(dplyr)
shows <- read_csv("tv_shows.csv")
disney <- read_csv("disney_plus_shows.csv")
netflix <- read_csv("netflix_titles.csv")
prime <- read_csv("Prime TV Shows Data set.csv")

Warning message:
"package 'rlist' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Parsed with column specification:
cols(
  ID = col_character(),
  Title = col_character(),
  Year = col_double(),
  Age = col_character(),
  IMDb = col_double(),
  Rotten_Tomatoes = col_character(),
  Netflix = col_double(),
  Hulu = col_double(),
  Prime_Video = col_double(),
  DisneyPlus = col_double(),
  type = col_double()
)
Warning message:
"85 parsing failures.
row col   expected    actual           file
 63  -- 11 columns 1 columns 'tv_shows.csv'
 69  -- 11 columns 1 columns 'tv_shows.csv'
423  -- 11 columns 1 columns 'tv_shows.csv'
453  -- 11 columns 1 columns 'tv_shows.csv'
528  -- 11 columns 1 columns 'tv_shows.csv'
... ... .......... ......... .......

Compondo a base do trabalho:

In [25]:
# Limpando o databse conjunto para os dados relevantes


cshows <- shows %>% mutate(Streamings = paste(sep = " ", ifelse(Netflix == 1, "Netflix", ""),
                                                         ifelse(Prime_Video == 1, "Prime", ""),
                                                         ifelse(DisneyPlus == 1, "Disney", "")) %>%
                                        trimws() %>%
                                        str_replace(" ",", ")) %>%
                                        select(-Netflix, -Hulu, -Prime_Video, -DisneyPlus, -type, -ID)

cshows <- subset(cshows, Streamings != "NA, NA NA" & Streamings != "")

# Importanto Genero dos demais databases

t1 <- netflix %>% mutate(Title = title, Genre = listed_in) %>% select(Title, Genre) 
t2 <- prime %>% mutate(Title = ) %>% select(Title, Genre)
t3 <- disney %>% mutate(Title = title, Genre = genre) %>% select(Title, Genre)
gb <- c("Title", "Genre")
t <- full_join(t1, full_join(t2, t3, by = gb), by = gb)

cshows <- left_join(cshows, t, by = 'Title')
glimpse(cshows)

Rows: 4,143
Columns: 7
$ Title           <chr> "Breaking Bad", "Stranger Things", "Money Heist", "She~
$ Year            <dbl> 2008, 2016, 2017, 2010, 2015, 2005, 2011, 2005, 2005, ~
$ Age             <chr> "18+", "16+", "18+", "16+", "18+", "16+", "18+", "16+"~
$ IMDb            <dbl> 9.5, 8.8, 8.4, 9.1, 8.7, 8.9, 8.8, 8.4, 8.4, 8.8, 9.2,~
$ Rotten_Tomatoes <chr> "96%", "93%", "91%", "78%", "97%", "81%", "83%", "93%"~
$ Streamings      <chr> "Netflix", "Netflix", "Netflix", "Netflix", "Netflix",~
$ Genre           <chr> "Crime TV Shows, TV Dramas, TV Thrillers", "TV Horror,~


Buscando a lista de gêneros

In [26]:
x <- ""
for(g in cshows[,'Genre'][[1]]){
    c <- str_replace_all(g,"TV","") %>%
         str_replace_all("Reality-", "Reality-Show") %>%
         str_replace_all("Sci-Fi", "Sci-fi") %>%
         str_replace_all("Sci-fi comedy", "") %>%
         str_replace_all(" &", ",") %>%
         str_replace_all("Movies", "") %>%
         str_replace_all("Shows", "") %>%
         str_replace_all("/", "") %>%
         str_replace_all("Mysteries", "Mystery") %>%
         str_replace_all("Comedies", "Comedy") %>%
         str_replace_all("Documentaries", "Documentary") %>%
         str_replace_all("Dramas", "Drama") %>%
         str_replace_all("Musicals", "Musical") %>%
         str_replace_all("Thrillers", "Thriller") %>%
         str_replace_all("Talk-Show", "") %>%
         str_replace_all("Romantic", "Romance") %>%
         str_replace_all("Anime Series", "Anime") %>%
         str_replace_all("Kids\'", "") %>%
         str_replace_all("Kids\'", "")

    x <- paste(x, trimws(c), sep = ",")
}

gens <- strsplit(x, ",")[[1]] %>%
trimws() %>%
unique()

gens

[1] ""                 "Crime"            "Drama"            "Thriller"        
 [5] "Horror"           "Mystery"          "Sci-fi"           "Fantasy"         
 [9] "NA"               "British"          "International"    "Comedy"          
[13] "Classic"          "Cult"             "Action"           "Adventure"       
[17] "Suspense"         "Anime"            "Romance"          "Spanish-Language"
[21] "Teen"             "Docuseries"       "Science"          "Nature"          
[25] "Reality"          "Documentary"      "Sports"           "Korean"          
[29] "Animation"        "Family"           "Musical"          "Independent"     
[33] "Stand-Up Comedy"  "Talk"             "Reality-Show"     "Game-Show"       
[37] "Kids"             "Children"         "Short"            "Music"           
[41] "Western"          "LGBTQ"            "Arts"             "Entertainment"   
[45] "Culture"

tirei

* shows
* movies
* separar por & (Sci-fi & Fantasy, Children & Family)
* alguns plurais

In [23]:
cshows


Title,Year,Age,IMDb,Rotten_Tomatoes,Streamings,Genre
Breaking Bad,2008,18+,9.5,96%,Netflix,"Crime TV Shows, TV Dramas, TV Thrillers"
Stranger Things,2016,16+,8.8,93%,Netflix,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy"
Money Heist,2017,18+,8.4,91%,Netflix,NA
Sherlock,2010,16+,9.1,78%,Netflix,"British TV Shows, Crime TV Shows, International TV Shows"
Better Call Saul,2015,18+,8.7,97%,Netflix,"Crime TV Shows, TV Comedies, TV Dramas"
The Office,2005,16+,8.9,81%,Netflix,Comedy
Black Mirror,2011,18+,8.8,83%,Netflix,"British TV Shows, International TV Shows, TV Dramas"
Supernatural,2005,16+,8.4,93%,Netflix,"Classic & Cult TV, TV Action & Adventure, TV Horror"
Supernatural,2005,16+,8.4,93%,Netflix,"Drama, Suspense, Horror"
Peaky Blinders,2013,18+,8.8,92%,Netflix,"British TV Shows, Crime TV Shows, International TV Shows"
